In [1]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time
import re
import shapefile
import os
key = os.environ.get('key')
client = ors.Client(key= key)


#Please note that due to the daily limit of the website and the limit of the number of reads per minute, it is recommended to read only ten rental data at a time
rent = pd.read_csv("../rent_data/rent_cleaned.csv")
rent = rent[1150:].reset_index(drop=True)


In [25]:
def cal_distance(coordinates, client):
    """
    function: calculate coordinates' distance
    param datas : durations, distances
    """
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    # api cannot read more than forty times per minute
    time.sleep(2.5)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds

In [70]:
summary = []
# CBD only have one corrdinate
for i in range(len(rent)):
    r_coor = []
    r_coor.append([rent["longitude"][i], rent["latitude"][i]])
    r_coor.append([144.946457, -37.840935])
    summary.append(r_coor)

In [27]:
def get_coordinates(summary):
    """
    function: Find the distances and durations that rach rental house drive to cbd
    param datas : rent_address, min_distance, min_duration
    """
    rent_address = []
    min_distance = []
    min_duration = []
    for i in range(len(summary)):
        distances = []
        durations = []
        matrix = cal_distance(summary[i], client)
        distances = distances + matrix[1][0][1:]
        durations = durations + matrix[0][0][1:]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][summary.index(summary[i])])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, min_distance, min_duration

In [71]:
# -- coding: utf-8 --
def data_frame(summary):
    """
    function: put the required columns into a dataframe
    param datas: dataframe
    """
    #get dataframe
    data = {'rent_address': get_coordinates(summary)[0], 'min_distance': get_coordinates(summary)[1], 'min_duration': get_coordinates(summary)[2]}
    df = DataFrame(data)
    return df

df = data_frame(summary)
df.to_csv("../data/raw/cbd distance and duration/cbd_distances_durations(8).csv", index=False)

In [2]:
# merge data
old = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(0).csv", index_col=0)
new = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(1).csv", index_col=0)

In [3]:
renew = pd.concat([old, new])
new2 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(2).csv", index_col=0)
renew2 = pd.concat([renew, new2])
new3 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(3).csv", index_col=0)
renew3 = pd.concat([renew2, new3])
new4 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(4).csv", index_col=0)
renew4 = pd.concat([renew3, new4])
new5 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(5).csv", index_col=0)
renew5 = pd.concat([renew4, new5])
new6 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(6).csv", index_col=0)
renew6 = pd.concat([renew5, new6])
new7 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(7).csv", index_col=0)
renew7 = pd.concat([renew6, new7])
new8 = pd.read_csv("../data/raw/cbd distance and duration/cbd_distances_durations(8).csv", index_col=0)
renew8 = pd.concat([renew7, new8]).reset_index()

In [4]:
# add rantal house's postcode and price
rent = pd.read_csv("../rent_data/rent_cleaned.csv")
new_data = renew8
postcode = []
for i in range(len(new_data)):
    postcode.append(new_data['rent_address'][i][-13:-9])
new_data['postcode'] = postcode
price =[]
for rent_a in new_data['rent_address']:
    price.append(rent[rent['listingSlug'] == rent_a]['price'].tolist()[0])
new_data['price'] = price
new_data = new_data.sort_values(by=["postcode"],ascending=True).reset_index(drop=True)

In [6]:
new_data.to_csv("../data/curated/cbd_distances_durations(final).csv")